<a href="https://colab.research.google.com/github/rjromo/generar_musica_python/blob/master/Generaci%C3%B3n__de__musica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [0]:
# Leyendo el archivo wav
sampling_freq, audio = wavfile.read('input_read.wav')


In [0]:
# Imprimiendo los parámetros
print ('\nShape:', audio.shape)
print ('Datatype:', audio.dtype)
print ('Duration:', round(audio.shape[0] / float(sampling_freq), 3), 'seconds')

In [0]:
# Normalizando los valores
audio = audio / (2.**15)

In [0]:
# Extraer los primeros 30 valores para graficar
audio = audio[:30]

In [0]:
# Construyendo el eje del tiempo
x_values = np.arange(0, len(audio), 1) / float(sampling_freq)
# Conviertiendo a segundos
x_values *= 1000

In [0]:
# Graficando la señal cortada
plt.plot(x_values, audio, color='black')
plt.xlabel('Tiempo (ms)')
plt.ylabel('Amplitud')
plt.title('Señal de Audio')
plt.show()

In [0]:
# Leyendo el archivo input_freq
sampling_freq, audio = wavfile.read('input_freq.wav')
# Normalizando los valores
audio = audio / (2.**15)
# Extraer longitud
len_audio = len(audio)

In [0]:
# Aplicando la transformada de Fourier
transformed_signal = np.fft.fft(audio)
half_length = int(np.ceil((len_audio + 1) / 2.0))
transformed_signal = abs(transformed_signal[0:half_length])
transformed_signal /= float(len_audio)
transformed_signal **= 2

In [0]:
# Extraer la longitud de la señal transformada
len_ts = len(transformed_signal)

# Casos pares e impares
if len_audio % 2:
    transformed_signal[1:len_ts] *= 2
else:
    transformed_signal[1:len_ts-1] *= 2


# Extrayendo la potencia en decibeles dB
power = 10 * np.log10(transformed_signal)

In [0]:
# Construyendo el eje del tiempo
x_values = np.arange(0, half_length, 1) * (sampling_freq / len_audio) / 1000.0

# Graficando
plt.figure()
plt.plot(x_values, power, color='black')
plt.xlabel('Freq (in kHz)')
plt.ylabel('Power (in dB)')
plt.show()

In [0]:
##Generando señales de audio

In [0]:
# Archivo que será guardado
output_file = 'output_generated.wav'

In [0]:
# Especificando los parámetros del audio
duration = 3  # segundos
sampling_freq = 44100  # Hz
tone_freq = 587
min_val = -2 * np.pi
max_val = 2 * np.pi

# Generar audio
t = np.linspace(min_val, max_val, duration * sampling_freq)
audio = np.sin(2 * np.pi * tone_freq * t)

# Añadir ruido
noise = 0.4 * np.random.rand(duration * sampling_freq)
audio += noise

In [0]:
# Escalar a 16 bits los valores enteros
scaling_factor = pow(2,15) - 1
audio_normalized = audio / np.max(np.abs(audio))
audio_scaled = np.int16(audio_normalized * scaling_factor)

# Escribir el archivo de salida
write(output_file, sampling_freq, audio_scaled)

In [0]:
# Extraer los primeros 100 valores
audio = audio[:100]

# Construyendo el eje del tiempo
x_values = np.arange(0, len(audio), 1) / float(sampling_freq)

# Convirtiendo a segundos
x_values *= 1000

# Graficando la señal
plt.plot(x_values, audio, color='black')
plt.xlabel('Tiempo (ms)')
plt.ylabel('Amplitud')
plt.title('Señal de Audio')
plt.show()


In [0]:
##Generando música


In [0]:
import json

In [0]:
# Sintetizar
def synthesizer(freq, duration, amp=1.0, sampling_freq=44100):
    # Construir el eje del tiempo
    t = np.linspace(0, duration, duration * sampling_freq)

    # Construir la señal de audio
    audio = amp * np.sin(2 * np.pi * freq * t)

    return audio.astype(np.int16) 

if __name__=='__main__':
    # Archivo json que contiene el mapa de las notas
    tone_map_file = 'tone_freq_map.json'
    
    # Leyendo el mapa de frecuencias
    with open(tone_map_file, 'r') as f:
        tone_freq_map = json.loads(f.read())
        
    # Configurando parámetros para la nota G
    input_tone = 'G'
    duration = 2     # seconds
    amplitude = 10000
    sampling_freq = 44100    # Hz

    # Generando el tono
    synthesized_tone = synthesizer(tone_freq_map[input_tone], duration, amplitude, sampling_freq)

    # Escribiendo al archivo de salida
    write('output_tone.wav', sampling_freq, synthesized_tone)